# Usando modelos pré-treinados

Iremos utilizar o Hugging Faces para facilitar a obtenção de modelos pré-treinados para fazermos o transfer learning

In [ ]:
!pip install transformers --quiet
!pip install datasets --quiet

In [ ]:
import torch.nn as nn
import torch

from dataloader import create_CIFAR10_loader
from tqdm import tqdm

In [ ]:
#Extraído de https://github.com/rasbt/stat453-deep-learning-ss21/blob/2202699c5fd38af398e2682f289a0868b1b91f0e/L13/code/helper_evaluation.py

def compute_history(model, data_loader, device, loss, n_samples = 50):

    with torch.no_grad():

        loss_ac, correct_pred, num_examples = 0, 0, 0

        for i, batch in enumerate(data_loader):
            features = batch['pixel_values'].to(device)
            targets = batch['labels'].to(device)
            
            output = model(features)

            logits = output['logits']

            loss_val = loss(logits, targets)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            loss_ac += (loss_val.item())
            correct_pred += (predicted_labels == targets.float()).sum()

            if i % n_samples == n_samples - 1:
              break
    return correct_pred.float()/num_examples * 100, loss_ac/num_examples * 100

## Carregando informações pré-treinadas

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

model_name = "google/vit-base-patch16-224"
model_name = "microsoft/resnet-50"

model = AutoModelForImageClassification.from_pretrained(model_name)

In [ ]:
epochs = 2 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_loader, val_loader = create_CIFAR10_loader(model_name, ID=True)

In [ ]:
minibatch_loss_list, train_acc_list, valid_acc_list, train_loss_list, valid_loss_list = [], [], [], [], []

criterion = torch.nn.functional.cross_entropy

for epoch in range(epochs):

    model.train()
    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        X_train = batch['pixel_values'].to(device)
        y_train = batch['labels'].to(device)

        output = model(X_train)
        logits = output['logits']
        loss = criterion(logits, y_train)
        
        loss.backward()
        optimizer.step()

        minibatch_loss_list.append(loss.item())
        
    #Validação
    model.eval()
    with torch.no_grad():
        train_acc, train_loss = compute_history(model, train_loader, device, criterion)
        valid_acc, valid_loss = compute_history(model, val_loader, device, criterion)
        
        train_acc_list.append(train_acc.item())
        valid_acc_list.append(valid_acc.item())
        
        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)


## Congelando algumas camadas para o ajuste da rede

In [ ]:
minibatch_loss_list, train_acc_list, valid_acc_list, train_loss_list, valid_loss_list = [], [], [], [], []

criterion = torch.nn.functional.cross_entropy

epochs = 2

for epoch in range(epochs):

    model.train()
    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        X_train = batch['pixel_values'].to(device)
        y_train = batch['labels'].to(device)

        output = model(X_train)
        logits = output['logits']
        loss = criterion(logits, y_train)
        
        loss.backward()
        optimizer.step()

        minibatch_loss_list.append(loss.item())

        
    #Validação
    model.eval()
    with torch.no_grad():
        train_acc, train_loss = compute_history(model, train_loader, device, criterion)
        valid_acc, valid_loss = compute_history(model, val_loader, device, criterion)
        
        train_acc_list.append(train_acc.item())
        valid_acc_list.append(valid_acc.item())
        
        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)
